## Extract the bug number for .java,v files. 
* We cannot get enough .java,v files from `jdt`, `platform`. We have to look at `jdt.core`. 
* How do we find a counterpart of `jdt.core` for `platform`? 

In [1]:
import os, re
import pandas as pd

# PATH_JDT = r"J:\EECS6444\Raw_Data\cvs_target\org.eclipse.jdt"
PATH_PLATFORM = r"J:\EECS6444\Raw_Data\cvs_target\org.eclipse.platform"
PATH_JDT_CORE = r"J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core"

def read_through_all_the_files_in_a_path(PATH):
    root_PATH = PATH.split("\\")[-1]
    with open("{}_no_pattern.csv".format(root_PATH), "w") as rp:
        rp.write("{},{}\n".format("file", "bugID"))
        group = os.walk(PATH)
        for path, directory, filelist in group:
            for file in filelist:
                try:
                    ## skip the non-java code: 
                    if ".java,v" not in file:
                            continue
                    ## get the lines from original files. 
                    lines = None
                    with open(os.path.join(path, file), "r", encoding="utf-8") as f:
                        lines = f.readlines()
                    ## skip the non-java file
                    ## this is the latest version of this file. 
                    # print(re.split("\s", lines[0].replace(";", ""))[1])
                    log_start = False
                    bugs = []
                    for l in lines:
                        line = l.strip().lower()
                        if log_start:
                            ## no pattern is used
                            rst_stage_1 = re.findall("(\d{4,})[\s]*", line)
                            bugs.extend(rst_stage_1)
                            ## some pattern is used
#                             if "fix" in line or "bug" in line: ## the condition here can be added. 
#                                 ## stage 1: the bug number will be like this: xxxx(space)(a series of numbers)(perhaps here are some other spaces)
#                                 rst_stage_1 = re.findall(" (\d{4,})[\s]*", line)
#                                 bugs.extend(rst_stage_1)
#                                 ## stage 2: we want to see if there is any bug id in url. 
#                                 urls = re.findall("https://bugs.eclipse.org/bugs[\s\S]*", line)
#                                 for url in urls:
#                                     rst_stage_2 = re.findall("(\d{4,})", url)
#                                     bugs.extend(rst_stage_2)
                        ## log starts from here.
                        if line == "log":
                            log_start = True
                        ## log ends here. 
                        elif line == "@":
                            log_start = False
                    for bug in bugs:
                        rp.write(
                            "{},{}\n".format(
                                root_PATH + os.path.join(path, file).replace(PATH, "").replace(",v", ""), 
                                bug
                            )
                        )
                except:
                    print("failed: {}".format(os.path.join(path, file)))
    return

read_through_all_the_files_in_a_path(PATH_PLATFORM) 
read_through_all_the_files_in_a_path(PATH_JDT_CORE) 

failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\compiler\org\eclipse\jdt\core\compiler\CharOperation.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\compiler\org\eclipse\jdt\core\compiler\InvalidInputException.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\compiler\org\eclipse\jdt\core\compiler\IProblem.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\compiler\org\eclipse\jdt\core\compiler\Attic\IScanner.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\compiler\org\eclipse\jdt\core\compiler\Attic\ITerminalSymbols.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\compiler\org\eclipse\jdt\internal\compiler\flow\UnconditionalFlowInfo.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\compiler\org\eclipse\jdt\internal\compiler\lookup\MethodVerifier15.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\compiler\org\eclipse\jdt\internal\compil

failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\model\org\eclipse\jdt\internal\core\util\InnerClassesAttributeEntry.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\model\org\eclipse\jdt\internal\core\util\LineNumberAttribute.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\model\org\eclipse\jdt\internal\core\util\LocalVariableAttribute.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\model\org\eclipse\jdt\internal\core\util\LocalVariableTableEntry.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\model\org\eclipse\jdt\internal\core\util\LRUCache.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\model\org\eclipse\jdt\internal\core\util\MethodInfo.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\model\org\eclipse\jdt\internal\core\util\ReferenceInfoAdapter.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\model\org\eclipse\jdt\internal\co

In [3]:
import os, re
import pandas as pd

# PATH_JDT = r"J:\EECS6444\Raw_Data\cvs_target\org.eclipse.jdt"
PATH_PLATFORM = r"J:\EECS6444\Raw_Data\cvs_target\org.eclipse.platform"
PATH_JDT_CORE = r"J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core"

def read_through_all_the_files_in_a_path_choose_pattern(PATH, pattern_used = False):
    """
    You can choose whether you want to use regular expression to extract the bugID pattern. 
    """
    root_PATH = PATH.split("\\")[-1]
    with open("{}_no_pattern.csv".format(root_PATH), "w") as rp:
        rp.write("{},{},{},{}\n".format("file", "fileVersion", "date","bugID"))
        group = os.walk(PATH)
        for path, directory, filelist in group:
            for file in filelist:
                try:
                    ## skip the non-java code:
                    if ".java,v" not in file:
                            continue
                    ## get the lines from original files.
                    lines = None
                    with open(os.path.join(path, file), "r", encoding="utf-8") as f:
                        lines = f.readlines()
                    ## skip the non-java file
                    ## this is the latest version of this file.
                    # print(re.split("\s", lines[0].replace(";", ""))[1])
                    log_start = False
                    version_extraction = True
                    version_start = False
                    ## the dict: versions is used to store the version info:
                    ## versions[a_certain_version] = {
                    ##     "date": None,
                    ##     "bugs": []
                    ## }
                    versions = {}
                    current_version = None
                    version_date = False
                    ##
                    for l in lines:
                        line = l.strip().lower()
                        ## starts from version_extraction, we start to extract version meta info
                        if version_extraction:
                            ## get the version line.
                            if line == "":
                                version_start = True
                            if line != "" and version_start == True:
                                version_start = False
                                current_version = line
                                versions[current_version] = {
                                    "date": None,
                                    "bugs": []
                                }
                                version_date = True
                            ## deal with date line. Some other meta info contained in this line as well. Parse them if needed. 
                            if "date" in line and version_date == True:
                                version_date = False
                                date = re.split(r"\s*", line)[1].replace(";", "")
                                versions[current_version]["date"] = date
                            ## end of version meta info zone 
                            if line == "desc":
                                version_start = False
                                version_extraction = False
                                current_version = None
                        ## lets' see if there is any version number line in recorded in versions. 
                        if line in versions:
                            current_version = line
                        ## add the bugIDs, the bugID will be stored according to version. 
                        if log_start:
                            ## no pattern is used
                            if not pattern_used:
                                rst_stage_1 = re.findall("(\d{4,})", line)
                                rst_stage_1 = [int(o) for o in rst_stage_1]
                                versions[current_version]["bugs"].extend(rst_stage_1)
                            ## some pattern is used
                            else:
                                if "fix" in line or "bug" in line: ## the condition here can be added.
                                    ## stage 1: the bug number will be like this: xxxx(space)(a series of numbers)(perhaps here are some other spaces)
                                    rst_stage_1 = re.findall(" (\d{4,})[\s]*", line)
                                    rst_stage_1 = [int(o) for o in rst_stage_1]
                                    versions[current_version]["bugs"].extend(rst_stage_1)
                                    ## stage 2: we want to see if there is any bug id in url.
                                    urls = re.findall("https://bugs.eclipse.org/bugs[\s\S]*", line)
                                    for url in urls:
                                        rst_stage_2 = re.findall("(\d{4,})", url)
                                        rst_stage_2 = [int(o) for o in rst_stage_2]
                                        versions[current_version]["bugs"].extend(rst_stage_2)
                        ## log starts from here.
                        if line == "log":
                            log_start = True
                        ## log ends here.
                        elif line == "@":
                            log_start = False
                    ## get rid of some useless stuff and then print the info. 
                    del versions["desc"]
                    for version in versions:
                        version_info = versions[version]
                        date = version_info["date"]
                        version_info["bugs"] = list(set(version_info["bugs"]))
                        for bug in version_info["bugs"]:
                            rp.write(
                                "{},{},{},{}\n".format(
                                    root_PATH + os.path.join(path, file).replace(PATH, "").replace(",v", ""),
                                    version,
                                    date,
                                    bug
                                )
                            )
                except:
                    print("failed: {}".format(os.path.join(path, file)))
    return

read_through_all_the_files_in_a_path_choose_pattern(PATH_PLATFORM)
read_through_all_the_files_in_a_path_choose_pattern(PATH_JDT_CORE)

C:\Users\xmk233\Anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\compiler\org\eclipse\jdt\core\compiler\CharOperation.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\compiler\org\eclipse\jdt\core\compiler\InvalidInputException.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\compiler\org\eclipse\jdt\core\compiler\IProblem.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\compiler\org\eclipse\jdt\core\compiler\Attic\IScanner.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\compiler\org\eclipse\jdt\core\compiler\Attic\ITerminalSymbols.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\compiler\org\eclipse\jdt\internal\compiler\flow\UnconditionalFlowInfo.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\compiler\org\eclipse\jdt\internal\compiler\lookup\MethodVerifier15.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\compiler\org\eclipse\jdt\internal\compil

2019年10月1日


~~Keep an eye on this: `swt-win32-3235.dll`, this shouldn't be in results.~~


~~Also keep an eye on this: https://bugs.eclipse.org/bugs/show_bug.cgi?id=196672 because we cannot extract this and the above at the same time.~~

In [7]:
import os, re
import pandas as pd

# PATH_JDT = r"J:\EECS6444\Raw_Data\cvs_target\org.eclipse.jdt"
PATH_PLATFORM = r"J:\EECS6444\Raw_Data\cvs_target\org.eclipse.platform"
PATH_JDT_CORE = r"J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core"

def read_through_all_the_files_to_see_lines_that_contains_number_series(PATH):
    root_PATH = PATH.split("\\")[-1]
    with open("{}_heheda.csv".format(root_PATH), "w") as rp:
        rp.write("{},{}\n".format("file", "bugID"))
        group = os.walk(PATH)
        for path, directory, filelist in group:
            for file in filelist:
                try:
                    ## skip the non-java code: 
                    if ".java,v" not in file:
                            continue
                    ## get the lines from original files. 
                    lines = None
                    with open(os.path.join(path, file), "r", encoding="utf-8") as f:
                        lines = f.readlines()
                    ## skip the non-java file
                    ## this is the latest version of this file. 
                    # print(re.split("\s", lines[0].replace(";", ""))[1])
                    log_start = False
                    bugs = []
                    for l in lines:
                        line = l.strip().lower()
                        if log_start:
                            ## no pattern is used
                            rst_stage_1 = re.findall("(\d{4,})[\s]*", line)
                            if re.findall("(\d{4,})", line):
                                with open("bug_lines_{}.txt".format(root_PATH), "a") as bl:
                                    bl.write("{}\n".format(l.strip()))
#                                 print(l.strip())
                            bugs.extend(rst_stage_1)
                            ## some pattern is used
                        ## log starts from here.
                        if line == "log":
                            log_start = True
                        ## log ends here. 
                        elif line == "@":
                            log_start = False
                    for bug in bugs:
                        rp.write(
                            "{},{}\n".format(
                                root_PATH + os.path.join(path, file).replace(PATH, "").replace(",v", ""), 
                                bug
                            )
                        )
                except:
                    print("failed: {}".format(os.path.join(path, file)))
    return

# read_through_all_the_files_to_see_lines_that_contains_number_series(PATH_PLATFORM) 
read_through_all_the_files_to_see_lines_that_contains_number_series(PATH_JDT_CORE) 

failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\compiler\org\eclipse\jdt\core\compiler\CharOperation.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\compiler\org\eclipse\jdt\core\compiler\InvalidInputException.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\compiler\org\eclipse\jdt\core\compiler\IProblem.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\compiler\org\eclipse\jdt\core\compiler\Attic\IScanner.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\compiler\org\eclipse\jdt\core\compiler\Attic\ITerminalSymbols.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\compiler\org\eclipse\jdt\internal\compiler\flow\UnconditionalFlowInfo.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\compiler\org\eclipse\jdt\internal\compiler\lookup\MethodVerifier15.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\compiler\org\eclipse\jdt\internal\compil

failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\model\org\eclipse\jdt\internal\core\util\ConstantPool.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\model\org\eclipse\jdt\internal\core\util\ConstantPoolEntry.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\model\org\eclipse\jdt\internal\core\util\ConstantValueAttribute.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\model\org\eclipse\jdt\internal\core\util\DefaultBytecodeVisitor.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\model\org\eclipse\jdt\internal\core\util\Disassembler.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\model\org\eclipse\jdt\internal\core\util\ExceptionAttribute.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\model\org\eclipse\jdt\internal\core\util\ExceptionTableEntry.java,v
failed: J:\EECS6444\Raw_Data\cvs\eclipse\org.eclipse.jdt.core\model\org\eclipse\jdt\internal\core\uti